# Les variables binaires en optimisation 
Tentative de clarification simplifiée, et étude par estelle derrien

<b>CREATION EN COURS </b>

- Valeurs : Elles prennent la valeur 0 ou 1 (non ou oui).
- Intérêt : Sélectionner une variable de décision , pénaliser une variable de décision, sélectionner une contrainte, discriminer une contrainte, établir une condition.
- Utilisation : Directement dans la fonction objectif, ou avec les contraintes.
- Déclaration : Elle diffère selon les solveurs, elle est plus facile dans certains.
- Semblable à : IF/THEN ( Si/Alors en Anglais ), Semblable à un principe ON/OFF pour les machines. Equivaut à un 'trigger' en Anglais, un déclencheur.
- Relaté à : Les méthodes <b>"Big M"</b>.
- Permettent souvent de : Eviter de faire une optimisation non linéaire.
- Permettent d'attribuer une <b>pénalité</b> à une variable de décision, si sa valeur dépasse un certain seuil.
- On évite d'utiliser la fonctionnalité de limites (Bounds) sur les variables de décision souvent fournies avec le solveur, <b>en même temps que</b> les variables binaires, c'est soit l'un,soit l'autre.

Liens :
http://www.yzuda.org/Useful_Links/optimization/if-then-else-02.html

https://benalexkeen.com/linear-programming-with-python-and-pulp-part-6/

https://stackoverflow.com/questions/58825442/how-can-i-write-an-if-condition-for-my-decision-variable-for-mixed-integer-linea

# Exemples simples:

Si la production de x dépasse 20, alors la contrainte z de coût maintenance s'applique, sinon, elle ne s'applique pas.

Si un objet X1(variable de décision) dépasse 20kgs, alors le container Z n'est pas utilisé, c'est le container Y qui est utilisé.

Si un élément chimique X1 (variable de décision) dépasse un seuil de 20grammes, alors, une pénalité lui est appliquée, ce qui privilégie l'élement chimique X2 dans l'optimisation finale.

Déclencher une machine X2 si la production sur la machine X1 est > 1000, sinon, ne pas la déclencher (Mode ON/OFF)



## 1/ Si la production de x dépasse 20, alors la contrainte z de coût maintenance s'applique, sinon, elle ne s'applique pas.

On s'entraine sur un programme linéaire simple avec Pulp .
La création de la variable binaire z se passe en deux temps
- 1. On instancie la variable binaire
- 2. On crée une contrainte qui mets en jeu cette variable binaire

Le problème : L'usine produit 2  objets x et y et les vends resp 10.5 euros et 8.5 euros, quand la production de l'objet x dépasse 20 unités, alors un coût de maintenance de 10 euros est soustrait à notre profit.  Comment modéliser ce problème simple avec les solveurs ?

Note : Ne fonctionne pas encore sous PULP

In [79]:
# Importer la librairie Pulp sous le pseudo p
import pulp as p 
  
# Créer un programme linéaire de maximisation
Lp_prob = p.LpProblem('Problem', p.LpMaximize)  

# -----------------------------------
# On définit nos constantes
#
# -----------------------------------

# On spécifie le cout de maintenance
cout_maintenance = 10

# On spécifie le seuil de déclenchement de la maintenance
declencheur_maintenance = 20

# -----------------------------------
# On définit nos variables de décision
#
# -----------------------------------
  
# On Crée les variables de décision du problème , x et y sont des objets que l'usine produit
x = p.LpVariable("x", lowBound = 0, cat='Integer')   # Create a variable x >= 0 
y = p.LpVariable("y", lowBound = 0, cat='Integer')   # Create a variable y >= 0 

# Comme on a besoin d'appliquer un cout de maintenance conditionnel, on a besoin 
# de définir une variable binaire qui va se déclencher si la production de x est supérieure
# à 20 objets !
z = p.LpVariable("z", lowBound=0, cat='Binary')
z.setInitialValue(0) # On tente d'initialiser la variable à 0

# -----------------------------------
# On définit la fonction objectif
#
# -----------------------------------
  
# Ecrire la fonction objectif à maximizer qui nous donne un résultat en Euros 
# Ici, x est vendu 10.5 euros et y 8.5 euros, le coût de 10 euros de maintenance
#  est soustrait seulement si z est positive.
Lp_prob +=  10.5 * x + 8.5 * y - cout_maintenance * z


# -----------------------------------
# On définit nos contraintes
#
# ----------------------------------- 

# Heures de travail au mois
# Ca prends 3 heures de crée un objet x, et 2 heures de créer un objet z
Lp_prob += 3 * x + 2 * y  <= 420

# Il faut produire au minimum ce nombre d'éléments  :
Lp_prob += x  >= 100
Lp_prob += y  >= 40

# La contrainte binaire qu'on doit faire:
# si x > 20 alors z = 1
# On utilise la méthode " BIgM "
M1 = 1e6
Lp_prob += z >= (x - 20)/M1


# -----------------------------------
# On résouds avec le solveur
#
# -----------------------------------

# Afficher le problème linéaire
# print(Lp_prob) 
status = Lp_prob.solve()   # Exécuter le solver
# print(p.LpStatus[status])   # Le statut de la solution

# Afficher la solution :
print(p.value(x),"Objets produits x")
print(p.value(y) , "Objets produits y"  )
print(p.value(z) , "La valeur de la variable binaire utilisée ou pas , 0 ou 1"  )
print(p.value(Lp_prob.objective) ,"est notre profit" )


100.0 Objets produits x
60.0 Objets produits y
1.0 La valeur de la variable binaire utilisée ou pas , 0 ou 1
1550.0 est notre profit


Maintenant, on va essayer avec le solveur DocPlex, qu ipermet d'écrire les contraintes binaires plus simplement

In [80]:

import cplex
import docplex.mp
from docplex.mp.model import Model

# On crée notre modèle
model = Model(name='LP_example', log_output=True)

# On crée nos variables de décision
x = model.integer_var(name='x')
y = model.integer_var(name='y')
z = model.binary_var(name='z')

# On crée la fonction objectif
model.maximize(10.5 * x + 8.5 * y - z * cout_maintenance)

# On crée les contraintes
model.add_constraint(x  >= 100)
model.add_constraint(y >= 40)
model.add_constraint(3 * x + 2 * y  <= 420)

# On spécifie le déclencheur de la contrainte z, 
# qui dit que si la production de x dépasse 20, 
# alors le coût de 10 euros est soustrait dans la fonction objectif
#if then constraint
model.add_constraint(model.if_then(x >= declencheur_maintenance, z == 1))


model.print_information() 
sol_model = model.solve()
model.print_solution()


Model: LP_example
 - number of variables: 4
   - binary=2, integer=2, continuous=0
 - number of constraints: 5
   - linear=3, indicator=1, equiv=1
 - parameters: defaults
 - objective: maximize
 - problem type is: MILP
Version identifier: 22.1.0.0 | 2022-03-25 | 54982fbec
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
MIP Presolve eliminated 4 rows and 2 columns.
MIP Presolve added 1 rows and 1 columns.
Reduced MIP has 1 rows, 3 columns, and 3 nonzeros.
Reduced MIP has 0 binaries, 3 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (0.01 ticks)
Found incumbent of value 1380.000000 after 0.01 sec. (0.01 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 1 rows and 1 columns.
MIP Presolve added 1 rows and 1 columns.
Reduced MIP has 1 rows, 3 columns, and 3 nonzeros.
Reduced MIP has 0 binaries, 3 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.00 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynam